# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

In [2]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm

In [3]:
data = pd.read_csv("../data/hulls_df_compact_matchday1.csv")
data = data.dropna()
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,561.031976,1195.207383
1,3.0,1.0,496.093264,1115.390332
2,4.0,1.0,466.225606,1083.424411
3,5.0,1.0,503.593453,1105.121644
4,6.0,1.0,563.030125,1132.199093


**NOTA:** provare a scalare i dati per vedere:
1. se il modello fitta più veloce
2. se i parametri finali sono più sensati

In [4]:
sequence = torch.tensor(data["HomeHull"].values)
hidden_dim = 2
sequence.shape

torch.Size([3911])

In [5]:
def model(sequence, hidden_dim, include_prior=True):
    length = len(sequence)
    with poutine.mask(mask=include_prior):
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(),
        )
        # Emission probabilities (1-dimensional for the area)
        probs_alpha = pyro.sample(
            "probs_alpha",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )

        probs_beta = pyro.sample(
            "probs_beta",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )
    
    x = 0  # Initial hidden state
    for t in pyro.markov(range(length)):
        x = pyro.sample(
            f"x_{t}",
            dist.Categorical(probs_x[x]),
            infer={"enumerate": "parallel"},
        )
        pyro.sample(
            f"y_{t}",
            dist.Gamma(probs_alpha[x], probs_beta[x]),
            obs=sequence[t],
        )
# Define the guide (variational distribution)
guide = AutoDelta(poutine.block(model, expose=["probs_x", "probs_alpha", "probs_beta"]))

---

**⚠️ DANGER ZONE ⚠️**

In [ ]:
pyro.clear_param_store()

---

In [6]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Training
num_steps = 1000
tqdm_bar = tqdm.tqdm(range(num_steps))
for step in tqdm_bar:
    loss = svi.step(sequence, hidden_dim)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 1000/1000 [39:58<00:00,  2.40s/it, LOSS=2.76e+4]


In [13]:
posterior = guide(sequence,hidden_dim)
# can i save posterior?
posterior

{'probs_x': tensor([[0.9631, 0.0369],
         [0.0634, 0.9366]], grad_fn=<ExpandBackward0>),
 'probs_alpha': tensor([  5.5652, 120.9643], grad_fn=<ExpandBackward0>),
 'probs_beta': tensor([0.0063, 0.1008], grad_fn=<ExpandBackward0>)}

In [14]:
torch.save(posterior,"parameters/singleHMM_home.pt")

In [21]:
mean_state1=5.5652/0.0063
print(f">> Mean of the Convex Hull for home team (STATE 1): {mean_state1:.2f} m^2")

>> Mean of the Convex Hull for home team (STATE 1): 883.37 m^2


In [20]:
mean_state2=120.9643/0.1008
print(f">> Mean of the Convex Hull for home team (STATE 2): {mean_state2:.2f} m^2")

>> Mean of the Convex Hull for home team (STATE 2): 1200.04 m^2


### Viterbi

In [8]:
posterior=torch.load("parameters/singleHMM_home.pt")
posterior

{'probs_x': tensor([[0.9631, 0.0369],
         [0.0634, 0.9366]], requires_grad=True),
 'probs_alpha': tensor([  5.5652, 120.9643], requires_grad=True),
 'probs_beta': tensor([0.0063, 0.1008], requires_grad=True)}

In [10]:
for name,value in posterior.items():
    pyro.param(name,value)

In [12]:
#marginals= elbo.compute_marginals(model,guide,sequence,hidden_dim) dopo 56 minuti ancora nada